In [15]:
"""
########################################################################################
# FAO - Events Visualization in Emergencies (EVE) - Data Extraction Script
########################################################################################

# Description:
This script programmatically downloads subsets of the EVE (Events Visualization in 
Emergencies) feature table hosted on ArcGIS Online (AGOL). It enables users to filter 
data by country and period, dynamically determine the most recent available period, 
and save the extracted data in either CSV or Excel format.

# Functionality:
- Connects to ArcGIS Online (AGOL) and retrieves flood monitoring data from EVE.
- Allows filtering by:
  - Country (`iso3`): Select a specific country using its ISO3 code or retrieve data 
    for all available countries.
  - Period (`period_number`):
    - `None`: Retrieve data for **all periods**.
    - An **integer**: Retrieve data for a **specific period**.
    - `"latest"`: Retrieve data for the **most recent available period**.
      - The latest period is dynamically determined based on FAO's period numbering 
        (two periods per month, starting from January 1, 2024).
      - If data for the expected latest period is unavailable, the script automatically 
        checks previous periods until it finds data.
- Excludes the `ObjectId` field from the extracted data.
- Outputs the data in **CSV** or **Excel** format with a meaningful filename.
- Saves the extracted data to a specified output folder.

# Output:
- The script saves the output file in the specified folder (`output_location`).
- The output filename follows the format:
  - If a period is selected: `filtered_data_<iso3>_<biweekly_group>_<timestamp>.csv`
  - If all periods are selected: `filtered_data_<iso3>_allperiods_<timestamp>.csv`
  - Example:
    - `filtered_data_BGD_Period_#27_(01-02-2025_to_15-02-2025)_20250221.csv`
    - `filtered_data_allcountries_allperiods_20250221.csv`

# About the FAO - Events Visualization in Emergencies (EVE) Tool:
- EVE is an FAO tool for **flood monitoring and analysis**, providing insights into 
  flood-affected areas and populations.
- Data outputs from EVE can be explored on the official dashboard:
  📌 **Dashboard:** https://data-in-emergencies.fao.org/apps/22e659f381fa41e5af05a67db001ac26/explore
- Additional information about the EVE system is available here:
  📌 **More Info:** https://data-in-emergencies.fao.org/pages/diem_eve

# Requirements:
- A **DIEM Account** is required to access and download data.  
  📌 **Register for a DIEM Account:** https://data-in-emergencies.fao.org/

########################################################################################
"""


import os
import datetime
from arcgis.gis import GIS
import pandas as pd

### USER DEFINED PARAMETERS 
USERNAME = ''  # Your DIEM username
PASS = ''  # Your DIEM password
# Output location (folder where the files will be saved)
output_location = r'C:\temp'  # Change this to your preferred location
# Data filters
iso3 = 'BGD'  # Set to desired ISO3 country code or leave empty for all EVE countries
period_number = "latest"  # Set to desired period number, None for all periods, or "latest" for the most recent available
file_format = 'CSV'  # Choose between 'CSV' or 'Excel'

# Get the current date and time
current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d%H%M%S")

# Calculate the latest period number dynamically
def calculate_current_period():
    start_year = 2024
    current_year = current_datetime.year
    current_month = current_datetime.month

    # Each year has 24 periods (2 per month)
    total_periods_since_start = (current_year - start_year) * 24 + (current_month - 1) * 2

    # Determine if we are in the first or second period of the current month
    if current_datetime.day >= 15:
        total_periods_since_start += 2  # Second period of the month
    else:
        total_periods_since_start += 1  # First period of the month

    return total_periods_since_start


# Feature table ID
FEATURE_TABLE_ID = '98265cbe82e344f88552d4c2d3615c3c'

# Initialize the where clause
where_clauses = []
if iso3:
    where_clauses.append(f"adm0_iso3 = '{iso3}'")

# Connect to AGOL
gis = GIS('https://hqfao-hub.maps.arcgis.com', USERNAME, PASS)

# Get the feature layer
item = gis.content.get(FEATURE_TABLE_ID)
if not item:
    raise ValueError("Feature table not found. Check the ID.")

feature_layer = item.tables[0]

# Determine the period to query
if period_number == "latest":
    period_number = calculate_current_period()
    while True:
        print(f"Trying period_number: {period_number}")
        query_result = feature_layer.query(where=f"period_number = {period_number}", out_fields="period_number", return_geometry=False, as_df=True)
        
        if not query_result.empty:
            print(f"Data found for period_number: {period_number}")
            break  # Exit loop when data is found
        else:
            print(f"No data for period_number: {period_number}, trying previous period...")
            period_number -= 1  # Try the previous period

if period_number is not None:
    where_clauses.append(f"period_number = {period_number}")

# Combine conditions or select all
where_clause = " AND ".join(where_clauses) if where_clauses else "1=1"

# Query the feature table but exclude the ObjectId field
fields = [f['name'] for f in feature_layer.properties.fields if f['name'].lower() != 'objectid']
df = feature_layer.query(where=where_clause, out_fields=",".join(fields), return_geometry=False, as_df=True)

# Rename 'people_affected' to 'people_exposed' if it exists
df.rename(columns={'pop_affected': 'pop_exposed'}, inplace=True)

# Determine filename components
iso3_label = iso3 if iso3 else "allcountries"

# If a period is selected, retrieve the biweekly_group value
if period_number is not None and not df.empty:
    biweekly_group = df['biweekly_group'].iloc[0].replace(" ", "_").replace("/", "-")  # Clean filename
    file_name = f"filtered_data_{iso3_label}_{biweekly_group}_{formatted_datetime}"
else:
    file_name = f"filtered_data_{iso3_label}_allperiods_{formatted_datetime}"

# Ensure output folder exists
os.makedirs(output_location, exist_ok=True)

# Define output file path
file_path = os.path.join(output_location, file_name)

# Save to chosen format
if file_format.upper() == 'CSV':
    df.to_csv(f"{file_path}.csv", index=False)
    print(f"Saved data as CSV: {file_path}.csv")
elif file_format.upper() == 'EXCEL':
    df.to_excel(f"{file_path}.xlsx", index=False)
    print(f"Saved data as Excel: {file_path}.xlsx")
else:
    print("Invalid file format. Please choose 'CSV' or 'Excel'.")

print("Process completed.")


Trying period_number: 28
No data for period_number: 28, trying previous period...
Trying period_number: 27
Data found for period_number: 27
Saved data as CSV: C:\temp\filtered_data_BGD_Period_#27_(01-02-2025_to_15-02-2025)_20250221120804.csv
Process completed.
